 _Lambda School Data Science Unit 2_
 
 # Classification & Validation Sprint Challenge

Follow the instructions for each numbered part to earn a score of 2. See the bottom of the notebook for a list of ways you can earn a score of 3.

#### For this Sprint Challenge, you'll predict whether a person's income exceeds $50k/yr, based on census data.

You can read more about the Adult Census Income dataset at the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/adult

#### Run this cell to load the data:

In [1]:
import numpy as np
import pandas as pd

columns = ['age', 
           'workclass', 
           'fnlwgt', 
           'education', 
           'education-num', 
           'marital-status', 
           'occupation', 
           'relationship', 
           'race', 
           'sex', 
           'capital-gain', 
           'capital-loss', 
           'hours-per-week', 
           'native-country', 
           'income']

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', 
                 header=None, names=columns)

df['income'] = df['income'].str.strip()

In [2]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
df = df.replace('?', np.NaN)

In [4]:
df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

In [5]:
df['income'].value_counts()

<=50K    24720
>50K      7841
Name: income, dtype: int64

## Part 1 — Begin with baselines

Split the data into an **X matrix** (all the features) and **y vector** (the target).

(You _don't_ need to split the data into train and test sets here. You'll be asked to do that at the _end_ of Part 1.)

In [6]:
import category_encoders as ce
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

X = df.drop(columns='income')
y = df['income']


What **accuracy score** would you get here with a **"majority class baseline"?** 
 
(You can answer this question either with a scikit-learn function or with a pandas function.)

In [7]:
y.value_counts(normalize=True)

<=50K    0.75919
>50K     0.24081
Name: income, dtype: float64

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)

In [13]:
y_train.value_counts(normalize=True)

<=50K    0.759173
>50K     0.240827
Name: income, dtype: float64

In [14]:
majority_class = y_train.mode()[0]
y_pred = [majority_class] * len(y_val)

In [15]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred)

0.7591810620601408

What **ROC AUC score** would you get here with a **majority class baseline?**

(You can answer this question either with a scikit-learn function or with no code, just your understanding of ROC AUC.)

In [16]:
from sklearn.metrics import roc_auc_score


pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    StandardScaler(), 
    LogisticRegression(solver='lbfgs', max_iter=1000)
)

pipeline.fit(X_train, y_train)
y_pred_proba = pipeline.predict_proba(X_val)[:,1]
roc_auc_score(y_val, y_pred_proba)

/home/seek/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/seek/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/seek/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:381: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


0.8989239207279731

In this Sprint Challenge, you will use **"Cross-Validation with Independent Test Set"** for your model validaton method.

First, **split the data into `X_train, X_test, y_train, y_test`**. You can include 80% of the data in the train set, and hold out 20% for the test set.

In [17]:
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((18233, 14), (7815, 14), (6513, 14), (18233,), (7815,), (6513,))

## Part 2 — Modeling with Logistic Regression!

- You may do exploratory data analysis and visualization, but it is not required.
- You may **use all the features, or select any features** of your choice, as long as you select at least one numeric feature and one categorical feature.
- **Scale your numeric features**, using any scikit-learn [Scaler](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) of your choice.
- **Encode your categorical features**. You may use any encoding (One-Hot, Ordinal, etc) and any library (category_encoders, scikit-learn, pandas, etc) of your choice.
- You may choose to use a pipeline, but it is not required.
- Use a **Logistic Regression** model.
- Use scikit-learn's [**cross_val_score**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function. For [scoring](https://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules), use **accuracy**.
- **Print your model's cross-validation accuracy score.**

In [18]:
X_train.describe(include='number')

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,18233.000000,1.823300e+04,18233.000000,18233.00000,18233.000000,18233.000000
mean,38.508583,1.892347e+05,10.071903,1084.23260,89.381506,40.403280
std,13.592388,1.043126e+05,2.580120,7548.82341,407.959492,12.348693
min,17.000000,1.228500e+04,1.000000,0.00000,0.000000,1.000000
25%,28.000000,1.186050e+05,9.000000,0.00000,0.000000,40.000000
50%,37.000000,1.785170e+05,10.000000,0.00000,0.000000,40.000000
75%,47.000000,2.363300e+05,12.000000,0.00000,0.000000,45.000000
max,90.000000,1.455435e+06,16.000000,99999.00000,4356.000000,99.000000


In [19]:
X_train_numeric = X_train.select_dtypes('number')
X_val_numeric   = X_val.select_dtypes('number')

In [29]:
X_train_encoded.sample(n=20)

,age,workclass_ Self-emp-inc,workclass_ Private,workclass_ Local-gov,workclass_ ?,workclass_ State-gov,workclass_ Federal-gov,workclass_ Self-emp-not-inc,workclass_ Without-pay,workclass_ Never-worked,...,native-country_ Canada,native-country_ Hungary,native-country_ Portugal,native-country_ Scotland,native-country_ Yugoslavia,native-country_ Outlying-US(Guam-USVI-etc),native-country_ Honduras,native-country_ Hong,native-country_ Cambodia,native-country_ Holand-Netherlands
114,36,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19478,36,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24357,26,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26734,30,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25294,34,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16354,35,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31792,32,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29903,30,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
598,25,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26093,62,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, X_train_imputed, y_train, cv=10)
print('Cross-Validation Accuracy Scores', scores)

Cross-Validation Accuracy Scores [0.79178082 0.7933114  0.79703785 0.79978058 0.7822271  0.79978058
 0.81294569 0.79155239 0.80087767 0.7975864 ]


In [31]:
from sklearn.preprocessing import MinMaxScaler

pipe = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(), 
    MinMaxScaler(), 
    LogisticRegression(solver='lbfgs', max_iter=1000)
)

scores = cross_val_score(pipe, X_train, y_train, cv=10)
print('Cross-Validation Accuracy Scores', scores)

Cross-Validation Accuracy Scores [0.85643836 0.83936404 0.83488755 0.85463522 0.84695557 0.85408667
 0.84366429 0.83708173 0.84750411 0.84421284]


## Part 3 — Modeling with Tree Ensembles!

Part 3 is the same as Part 2, except this time, use a **Random Forest** or **Gradient Boosting** classifier. You may use scikit-learn, xgboost, or any other library. Then, print your model's cross-validation accuracy score.

In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

models = [LogisticRegression(solver='lbfgs', max_iter=1000), 
          DecisionTreeClassifier(max_depth=3), 
          DecisionTreeClassifier(max_depth=None), 
          RandomForestClassifier(max_depth=3, n_estimators=100, n_jobs=-1, random_state=42), 
          RandomForestClassifier(max_depth=None, n_estimators=100, n_jobs=-1, random_state=42), 
          XGBClassifier(max_depth=3, n_estimators=100, n_jobs=-1, random_state=42)]

for model in models:
    print(model, '\n')
    score = cross_val_score(model, X_train_numeric, y_train, scoring='accuracy', cv=5).mean()
    print('Cross-Validation Accuracy:', score, '\n', '\n')

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False) 

Cross-Validation Accuracy: 0.7988818135379543 
 

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best') 

Cross-Validation Accuracy: 0.8049687231501952 
 

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0

## Part 4 — Calculate classification metrics from a confusion matrix

Suppose this is the confusion matrix for your binary classification model:

<table>
  <tr>
    <td colspan="2" rowspan="2"></td>
    <td colspan="2">Predicted</td>
  </tr>
  <tr>
    <td>Negative</td>
    <td>Positive</td>
  </tr>
  <tr>
    <td rowspan="2">Actual</td>
    <td>Negative</td>
    <td style="border: solid">85</td>
    <td style="border: solid">58</td>
  </tr>
  <tr>
    <td>Positive</td>
    <td style="border: solid">8</td>
    <td style="border: solid"> 36</td>
  </tr>
</table>

Calculate accuracy

In [39]:
true_negative = 85
false_positive = 58
false_negative = 8
true_positive = 36
predicted_negative = true_negative + false_negative
predicted_positive = false_positive + true_positive

accuracy = (true_positive + true_negative) / (predicted_negative + predicted_positive)

print('Accuracy', accuracy)

Accuracy 0.6470588235294118


Calculate precision

In [40]:
precision = true_positive / predicted_positive

print('Precision', precision)

Precision 0.3829787234042553


Calculate recall

In [41]:
actual_positive = predicted_negative + false_positive

recall = true_positive / actual_positive

print('Recall', recall)

Recall 0.23841059602649006


## BONUS — How you can earn a score of 3

### Part 1
Do feature engineering, to try improving your cross-validation score.

### Part 2
Experiment with feature selection, preprocessing, categorical encoding, and hyperparameter optimization, to try improving your cross-validation score.

### Part 3
Which model had the best cross-validation score? Refit this model on the train set and do a final evaluation on the held out test set — what is the test score? 

### Part 4
Calculate F1 score and False Positive Rate. 